# Restart

## Импорт библиотек

In [ ]:
!pip install moexalgo


In [ ]:
import pandas as pd
import plotly.graph_objects as go

from moexalgo import Ticker
from datetime import datetime


## Config


In [ ]:
# Списки тикеров акций в составе индекса MOEX в разные промежутки между ребалансировками:
tickers___01_01_2023___16_03_2023_ = [
    "GAZP", "SBER", "SBERP", "LKOH", "GMKN", "NVTK", "YNDX", "ROSN",
    "MGNT", "TCSG", "TATN", "PHOR", "PLZL", "SNGS", "MTSS", "FIVE",
    "ALRS", "CHMF", "SNGSP", "NLMK", "MOEX", "IRAO", "POLY", "OZON",
    "RUAL", "PIKK", "VTBR", "MAGN", "VKCO", "RTKM", "TATNP", "TRNFP",
    "HYDR", "CBOM", "FIXP", "AFKS", "ENPG", "GLTR", "AFLT"
] # +DSKY, при парсинге с Мосбиржи падает ошибка, поэтому создадим список без него

tickers___01_01_2023___16_03_2023 = [
    "GAZP", "SBER", "SBERP", "LKOH", "GMKN", "NVTK", "YNDX", "ROSN",
    "MGNT", "TCSG", "TATN", "PHOR", "PLZL", "SNGS", "MTSS", "FIVE",
    "ALRS", "CHMF", "SNGSP", "NLMK", "MOEX", "IRAO", "POLY", "OZON",
    "RUAL", "PIKK", "VTBR", "MAGN", "VKCO", "RTKM", "TATNP", "TRNFP",
    "HYDR", "CBOM", "FIXP", "AFKS", "ENPG", "GLTR", "AFLT", "DSKY"
] # список с Детским миром

tickers___17_03_2023___15_06_2023 = [
    "GAZP", "LKOH", "SBER", "SBERP", "GMKN", "NVTK", "YNDX", "ROSN",
    "MGNT", "TATN", "PLZL", "TCSG", "SNGS", "MTSS", "CHMF", "PHOR",
    "NLMK", "SNGSP", "ALRS", "POLY", "MOEX", "IRAO", "FIVE", "PIKK",
    "RUAL", "OZON", "MAGN", "VTBR", "CBOM", "RTKM", "TATNP", "TRNFP",
    "VKCO", "AFKS", "HYDR", "AFLT", "ENPG", "FIXP", "AGRO", "GLTR"
]

tickers___16_06_2023___21_09_2023 = [
    "LKOH", "GAZP", "SBER", "SBERP", "GMKN", "TATN", "TATNP", "SNGS",
    "SNGSP", "MGNT", "NVTK", "PLZL", "ROSN", "PIKK", "IRAO", "CHMF",
    "MTSS", "YNDX", "RUAL", "NLMK", "ALRS", "MAGN", "PHOR", "RTKM",
    "VTBR", "TCSG", "AFLT", "TRNFP", "OZON", "AFKS", "AGRO", "FEES",
    "CBOM", "POLY", "VKCO", "ENPG", "SGZH", "MOEX", "FIVE", "HYDR",
    "GLTR", "FIXP"
]

tickers___22_09_2023___21_12_2023 =[
    "LKOH", "GAZP", "SBER", "GMKN", "TATN", "NVTK", "SNGS", "PLZL",
    "SNGSP", "SBERP", "ROSN", "PIKK", "RUAL", "CHMF", "NLMK", "MGNT",
    "IRAO", "YNDX", "ALRS", "MAGN", "MTSS", "VTBR", "RTKM", "MOEX",
    "TATNP", "PHOR", "OZON", "ENPG", "AGRO", "TCSG", "CBOM", "AFLT",
    "AFKS", "TRNFP", "FEES", "VKCO", "FIVE", "SELG", "SGZH", "QIWI",
    "POSI", "HYDR", "UPRO", "POLY", "GLTR"
]

tickers___22_12_2023___29_12_2023 =[
    "LKOH", "SBER", "SBERP", "GAZP", "GMKN", "TATN", "TATNP", "NVTK",
    "SNGS", "SNGSP", "ROSN", "PLZL", "RUAL", "CHMF", "MGNT", "PIKK",
    "IRAO", "NLMK", "YNDX", "MAGN", "ALRS", "MTSS", "MOEX", "RTKM",
    "VTBR", "AGRO", "OZON", "FIVE", "PHOR", "CBOM", "TCSG", "FLOT",
    "TRNFP", "MTLR", "AFKS", "AFLT", "FEES", "ENPG", "MTLRP", "VKCO",
    "SMLT", "MSNG", "POSI", "SELG", "POLY", "UPRO", "HYDR", "QIWI",
    "SGZH", "GLTR"
]


start_date = datetime(2022, 10, 1)
end_date = datetime(2023, 12, 31)
start_date_IMOEX = datetime(2023, 1, 1)


##  Парсинг цен на акции, входивщие в индекс Мосбиржи

In [ ]:
# Объединение списков во множество для получения уникальных тикеров
tickers_set = set(tickers___01_01_2023___16_03_2023_ +
                 tickers___17_03_2023___15_06_2023 +
                 tickers___16_06_2023___21_09_2023 +
                 tickers___22_09_2023___21_12_2023 +
                 tickers___22_12_2023___29_12_2023 )

tickers = list(tickers_set)


In [ ]:
# Создадим дата-фрейм для хранения меток превышение цены над 50-MA
higher_50_MA = pd.DataFrame()

for ticker in tickers:
    # Получаем данные с Мосбиржи
    t = Ticker(ticker)
    df_ticker = t.candles(date=start_date, till_date=end_date, period='1D')
    # Переименовываем столбец с датой
    df_ticker = df_ticker.rename(columns={'begin': 'date'})

    # Вычислим значение 50-дневной скользящей средней и удаляем строки с датой раньше 2023 года
    df_ticker['50_MA'] = df_ticker['close'].rolling(window=50).mean()
    df_ticker = df_ticker[df_ticker['date'].dt.year >= 2023]

    # Если текущее значение цены больше 50-дневной средней, то ставим метку 1
    df_ticker.loc[df_ticker['close'] > df_ticker['50_MA'], 'higher_50_MA'] = 1
    df_ticker.loc[df_ticker['close'] <= df_ticker['50_MA'], 'higher_50_MA'] = 0

    # Удалим лишние поля и поменяем порядок полей
    df_ticker = df_ticker.drop(columns=['value', 'volume', 'end', 'open', 'high', 'low'])
    columns_copy = ['date', 'close', '50_MA', 'higher_50_MA']
    df_ticker = df_ticker[columns_copy].copy()

    # Добавим дату в дата-фрейм с метками
    if 'date' not in higher_50_MA.columns:
        higher_50_MA['date'] = df_ticker['date']

    # Сохраняем столбец с метками из текущего дата-фрейма в отдельный, если количество торговых дней совпадало
    if len(higher_50_MA) == len(df_ticker):
        ticker_values = df_ticker['higher_50_MA']
        higher_50_MA[f'{ticker}'] = ticker_values
    # Иначе сохраним дата-фрейм, чтобы его объединить другим методом
    else:
        print(f'Не получилось добавить информацию по {ticker}')
        globals()[f"{ticker}"] = df_ticker


Не получилось добавить информацию по FEES


<ipython-input-548-8146da5b82eb>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Не получилось добавить информацию по POLY
Не получилось добавить информацию по VKCO


In [ ]:
# Добавим данные из дата-фреймов, в которых количество строк отличается от
# большинства более сложным объединением по дате
FEES_subset = FEES[['date', 'higher_50_MA']]
higher_50_MA = pd.merge(higher_50_MA, FEES_subset, on='date', how='left')
higher_50_MA = higher_50_MA.rename(columns={'higher_50_MA': 'FEES'})

POLY_subset = POLY[['date', 'higher_50_MA']]
higher_50_MA = pd.merge(higher_50_MA, POLY_subset, on='date', how='left')
higher_50_MA = higher_50_MA.rename(columns={'higher_50_MA': 'POLY'})

VKCO_subset = VKCO[['date', 'higher_50_MA']]
higher_50_MA = pd.merge(higher_50_MA, VKCO_subset, on='date', how='left')
higher_50_MA = higher_50_MA.rename(columns={'higher_50_MA': 'VKCO'})

# Заменяем все значения NaN на 0
higher_50_MA.fillna(0, inplace=True)

## Отдельный парсинг Детского Мира


In [ ]:
# Загружаем данные, которые скачал с сайта Investing.com
file_path = '/content/drive/My Drive/Соревнования/Headlines/DSKY.csv'
df_DSKY = pd.read_csv(file_path)

# Меняем порядок строк наоборот
DSKY = df_DSKY[::-1]

# Переименовываем нужные поля и удаляем лишние
DSKY = DSKY.rename(columns={'Дата': 'date'})
DSKY = DSKY.rename(columns={'Цена': 'close'})
DSKY = DSKY.drop(columns=['Откр.', 'Макс.', 'Мин.', 'Объём', 'Изм. %'])

# Меняем типы полей с object на верные
DSKY['date'] = pd.to_datetime(DSKY['date'], format='%d.%m.%Y')
DSKY['close'] = DSKY['close'].str.replace(',', '.').astype(float)

# Вычисляем значение 50-дневной скользящей средней и удаляем строки с датой раньше 2023 года
DSKY['50_MA'] = DSKY['close'].rolling(window=50).mean()
DSKY = DSKY[DSKY['date'].dt.year >= 2023]

# Если текущее значение цены больше 50-дневной средней, то ставим метку 1
DSKY['higher_50_MA'] = DSKY.apply(lambda row: 1 if row['close'] > row['50_MA'] else 0, axis=1)
DSKY['higher_50_MA'] = DSKY['higher_50_MA'].astype(int)
DSKY = DSKY.drop(columns=['close'])

# Сохраняем столбец с метками из текущего дата-фрейма в ранее созданный с остальными компаниями
higher_50_MA['DSKY'] = DSKY['higher_50_MA']


## Парсинг индекса Мосбиржи

In [ ]:
IMOEX = Ticker('IMOEX')
df_IMOEX = IMOEX.candles(date=start_date_IMOEX, till_date=end_date, period='1D')
df_IMOEX = df_IMOEX.drop(columns=['end'])
df_IMOEX = df_IMOEX.rename(columns={'begin': 'date'})


## Вычисление доли акций выше 50MA в индексе Мосбиржи

In [ ]:
# Инциализируем дата-фреймы, в которых посчитаем долю акций выше 50MA
# для разных интервалов между ребалансировками индекса MOEX
higher_50MA___01_01_2023___16_03_2023 = pd.DataFrame()
higher_50MA___17_03_2023___15_06_2023 = pd.DataFrame()
higher_50MA___16_06_2023___21_09_2023 = pd.DataFrame()
higher_50MA___22_09_2023___21_12_2023 = pd.DataFrame()
higher_50MA___22_12_2023___29_12_2023 = pd.DataFrame()

# Скопируем только нужные столбцы акций для конкретного периода между ребалнсировками
columns_copy___01_01_2023___16_03_2023 = [col for col in higher_50_MA.columns if col in tickers___01_01_2023___16_03_2023]
higher_50MA___01_01_2023___16_03_2023 = higher_50_MA.loc[:, columns_copy___01_01_2023___16_03_2023].copy()

columns_copy___17_03_2023___15_06_2023 = [col for col in higher_50_MA.columns if col in tickers___17_03_2023___15_06_2023]
higher_50MA___17_03_2023___15_06_2023 = higher_50_MA.loc[:, columns_copy___17_03_2023___15_06_2023].copy()

columns_copy___16_06_2023___21_09_2023 = [col for col in higher_50_MA.columns if col in tickers___16_06_2023___21_09_2023]
higher_50MA___16_06_2023___21_09_2023 = higher_50_MA.loc[:, columns_copy___16_06_2023___21_09_2023].copy()

columns_copy___22_09_2023___21_12_2023 = [col for col in higher_50_MA.columns if col in tickers___22_09_2023___21_12_2023]
higher_50MA___22_09_2023___21_12_2023 = higher_50_MA.loc[:, columns_copy___22_09_2023___21_12_2023].copy()

columns_copy___22_12_2023___29_12_2023 = [col for col in higher_50_MA.columns if col in tickers___22_12_2023___29_12_2023]
higher_50MA___22_12_2023___29_12_2023 = higher_50_MA.loc[:, columns_copy___22_12_2023___29_12_2023].copy()


# Рассчитаем для каждого дата-фрейма новое поле для доли компаний, которые торгуется выше 50MA
higher_50MA___01_01_2023___16_03_2023['fraction_higher_MA'] = (higher_50MA___01_01_2023___16_03_2023.sum(axis=1) / len(tickers___01_01_2023___16_03_2023))
higher_50MA___17_03_2023___15_06_2023['fraction_higher_MA'] = (higher_50MA___01_01_2023___16_03_2023.sum(axis=1) / len(tickers___17_03_2023___15_06_2023))
higher_50MA___16_06_2023___21_09_2023['fraction_higher_MA'] = (higher_50MA___01_01_2023___16_03_2023.sum(axis=1) / len(tickers___16_06_2023___21_09_2023))
higher_50MA___22_09_2023___21_12_2023['fraction_higher_MA'] = (higher_50MA___01_01_2023___16_03_2023.sum(axis=1) / len(tickers___22_09_2023___21_12_2023))
higher_50MA___22_12_2023___29_12_2023['fraction_higher_MA'] = (higher_50MA___01_01_2023___16_03_2023.sum(axis=1) / len(tickers___22_12_2023___29_12_2023))

# Скопируем поле даты в полученные дата-фреймы
higher_50MA___01_01_2023___16_03_2023['date'] = higher_50_MA['date']
higher_50MA___17_03_2023___15_06_2023['date'] = higher_50_MA['date']
higher_50MA___16_06_2023___21_09_2023['date'] = higher_50_MA['date']
higher_50MA___22_09_2023___21_12_2023['date'] = higher_50_MA['date']
higher_50MA___22_12_2023___29_12_2023['date'] = higher_50_MA['date']

# Оставим только столбец с датой и столбец с долей компаний торгующихся выше 50MA
higher_50MA___01_01_2023___16_03_2023 = higher_50MA___01_01_2023___16_03_2023[['date', 'fraction_higher_MA']]
higher_50MA___17_03_2023___15_06_2023 = higher_50MA___17_03_2023___15_06_2023[['date', 'fraction_higher_MA']]
higher_50MA___16_06_2023___21_09_2023 = higher_50MA___16_06_2023___21_09_2023[['date', 'fraction_higher_MA']]
higher_50MA___22_09_2023___21_12_2023 = higher_50MA___22_09_2023___21_12_2023[['date', 'fraction_higher_MA']]
higher_50MA___22_12_2023___29_12_2023 = higher_50MA___22_12_2023___29_12_2023[['date', 'fraction_higher_MA']]

# Обрежем построчно дата-фреймы по их временным промежуткам между датами ребалнсировок
higher_50MA___01_01_2023___16_03_2023 = higher_50MA___01_01_2023___16_03_2023[(higher_50MA___01_01_2023___16_03_2023['date'] >= '2023-01-01') & (higher_50MA___01_01_2023___16_03_2023['date'] <= '2023-03-16')]
higher_50MA___17_03_2023___15_06_2023 = higher_50MA___17_03_2023___15_06_2023[(higher_50MA___17_03_2023___15_06_2023['date'] >= '2023-03-17') & (higher_50MA___17_03_2023___15_06_2023['date'] <= '2023-06-15')]
higher_50MA___16_06_2023___21_09_2023 = higher_50MA___16_06_2023___21_09_2023[(higher_50MA___16_06_2023___21_09_2023['date'] >= '2023-06-16') & (higher_50MA___16_06_2023___21_09_2023['date'] <= '2023-09-21')]
higher_50MA___22_09_2023___21_12_2023 = higher_50MA___22_09_2023___21_12_2023[(higher_50MA___22_09_2023___21_12_2023['date'] >= '2023-09-22') & (higher_50MA___22_09_2023___21_12_2023['date'] <= '2023-12-21')]
higher_50MA___22_12_2023___29_12_2023 = higher_50MA___22_12_2023___29_12_2023[(higher_50MA___22_12_2023___29_12_2023['date'] >= '2023-12-22') & (higher_50MA___22_12_2023___29_12_2023['date'] <= '2023-12-29')]


In [ ]:
# Объединим информацию из полученных выше дата-фреймов с данными индекса MOEX, объединив по полю date
df_IMOEX = pd.merge(df_IMOEX, higher_50MA___01_01_2023___16_03_2023, on='date', how='left', suffixes=('', '_1'))
df_IMOEX = pd.merge(df_IMOEX, higher_50MA___17_03_2023___15_06_2023, on='date', how='left', suffixes=('', '_2'))
df_IMOEX = pd.merge(df_IMOEX, higher_50MA___16_06_2023___21_09_2023, on='date', how='left', suffixes=('', '_3'))
df_IMOEX = pd.merge(df_IMOEX, higher_50MA___22_09_2023___21_12_2023, on='date', how='left', suffixes=('', '_4'))
df_IMOEX = pd.merge(df_IMOEX, higher_50MA___22_12_2023___29_12_2023, on='date', how='left', suffixes=('', '_5'))

# Соберём все значения fraction_higher_MA_1-5 в одно поле, добавим колонку для процентов и удалим лишние поля
fields_to_sum = ['fraction_higher_MA', 'fraction_higher_MA_2', 'fraction_higher_MA_3', 'fraction_higher_MA_4', 'fraction_higher_MA_5']
df_IMOEX['fraction_higher_50MA'] = df_IMOEX[fields_to_sum].sum(axis=1)
df_IMOEX['percentages_higher_50MA'] = df_IMOEX['fraction_higher_50MA'] * 100
df_IMOEX = df_IMOEX.drop(columns=['value', 'volume', 'fraction_higher_MA_2', 'fraction_higher_MA_3', 'fraction_higher_MA_4', 'fraction_higher_MA_5'])

# Поменяем колонки местами для большего порядака
columns_copy = ['date', 'open', 'close',	'high',	'low', 'fraction_higher_50MA', 'percentages_higher_50MA']
df_IMOEX = df_IMOEX[columns_copy].copy()


In [ ]:
# Сохранение дата-фрейма в файлы
df_IMOEX.to_excel("IMOEX.xlsx", index=False)
df_IMOEX.to_csv("IMOEX.csv", index=False)


In [ ]:
df_IMOEX


,date,open,close,high,low,fraction_higher_50MA,percentages_higher_50MA
0,2023-01-03,2157.18,2172.68,2174.23,2157.18,0.5000,50.00
1,2023-01-04,2171.54,2168.42,2179.56,2162.06,0.5000,50.00
2,2023-01-05,2170.40,2156.67,2171.94,2154.16,0.5000,50.00
3,2023-01-06,2157.32,2156.39,2160.08,2153.32,0.5000,50.00
4,2023-01-09,2163.43,2163.50,2169.71,2162.01,0.5500,55.00
...,...,...,...,...,...,...,...
249,2023-12-25,3095.42,3099.02,3108.14,3086.89,0.2870,28.70
250,2023-12-26,3090.39,3094.72,3098.69,3080.92,0.2665,26.65
251,2023-12-27,3100.99,3097.49,3112.19,3097.31,0.2460,24.60
252,2023-12-28,3099.56,3101.99,3101.99,3081.98,0.2665,26.65


## Визуализация

In [ ]:
fig = go.Figure()
# Закрашиваем область под графиком IMOEX
fig.add_trace(go.Scatter(x=df_IMOEX['date'], y=df_IMOEX['close'], mode='lines',
                         fill='tozeroy', fillcolor='rgba(255, 255, 255, 0.1)', showlegend=False))

# Добавляем график индекса
fig.add_trace(go.Scatter(x=df_IMOEX['date'], y=df_IMOEX['close'], mode='lines', name='IMOEX', line=dict(color='red', width=2)))

# Добавляем график доли компаний выше MA
fig.add_trace(go.Scatter(x=df_IMOEX['date'], y=df_IMOEX['fraction_higher_50MA'], mode='lines',
                         name='Доля компаний, торгующихся выше 50-дневной скользящей средней (%)', line=dict(color='green', width=2), yaxis='y2'))

fig.update_layout(template='plotly_dark',
                   xaxis=dict(rangeslider=dict(visible=True), type='date'),
                   yaxis=dict(color='red', side='left', position=0.005, tickfont=dict(size=15)),
                   yaxis2=dict(color='green', overlaying='y', side='right', tickformat=".0%", showgrid=False, tickfont=dict(size=15)),
                   legend=dict(x=0, y=1.1, orientation='h', font=dict(size=18)),
                   height=800)

fig.show()


In [ ]:
fig = go.Figure()
# Закрашиваем область под графиком IMOEX
fig.add_trace(go.Scatter(x=df_IMOEX['date'], y=df_IMOEX['close'], mode='lines',
                         fill='tozeroy', fillcolor='rgba(255, 0, 0, 0.1)', showlegend=False))

# Добавляем график индекса
fig.add_trace(go.Scatter(x=df_IMOEX['date'], y=df_IMOEX['close'], mode='lines', name='IMOEX', line=dict(color='red', width=2)))

# Добавляем график доли компаний выше MA
fig.add_trace(go.Scatter(x=df_IMOEX['date'], y=df_IMOEX['fraction_higher_50MA'], mode='lines',
                         name='Доля компаний, торгующихся выше 50-дневной скользящей средней (%)', line=dict(color='green', width=2), yaxis='y2'))

fig.update_layout(template='plotly',
                   xaxis=dict(rangeslider=dict(visible=True), type='date'),
                   yaxis=dict(color='red', side='left', tickfont=dict(size=15)),
                   yaxis2=dict(color='green', overlaying='y', side='right', tickformat=".0%", showgrid=False, tickfont=dict(size=15)),
                   legend=dict(x=0, y=1.1, orientation='h', font=dict(size=18)),
                   height=800)

fig.show()
